In [9]:
from datetime import datetime
import time
import pandas as pd
import yaml
from yaml.loader import SafeLoader

fname='Milestone1A.yaml'

with open(fname) as f:
    conf = yaml.load(f, Loader=SafeLoader)
print(conf)

{'M1A_Workflow': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskA': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskA_Input', 'ExecutionTime': '1'}}, 'TaskB': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskB_Input', 'ExecutionTime': '2'}}, 'FlowA': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskC': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskC_Input', 'ExecutionTime': '3'}}, 'TaskD': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskD_Input', 'ExecutionTime': '4'}}}}}}}


In [10]:
def TimeFunction(FunctionInput,ExecutionTime):
    print(1)
    time.sleep(int(ExecutionTime))
    print(int(ExecutionTime))
    return
    
def DataLoad(Filename):
    print(2)
    DataTable=pd.read_csv(Filename)
    NoOfDefects=DataTable.shape[0]-1    
    return DataTable, NoOfDefects

def callfunction(name,inputs):
    if name=='TimeFunction':
        TimeFunction(inputs[0],inputs[1])
    elif name=='DataLoad':
        DataLoad(inputs[0])

In [11]:
def runTask(task,b_list):
    print(task)
    name=''
    for i in b_list:
        name+=str(i)+'.'
    
    f.write(str(datetime.now())+';'+name[:-1]+' Entry\n')
    if task['Type']=='Task':
        funs=''
        params=''
        inputs=[]

        for i in task['Inputs'].values():
            inputs.append(i)
            funs+=str(i)+','
        if(task['Function']=='TimeFunction'):
            params=str(inputs[-1])+','
        else:
            params=funs
                        
        f.write(str(datetime.now())+';'+name[:-1]+' Executing ' +str(task['Function'])+' ('+params[:-1]+')\n')
        callfunction(task['Function'],inputs)
        
    else:
        Act=task['Activities']
        for i in Act:
            par=b_list
            par.append(i)
            runTask(Act[i],par)
    f.write(str(datetime.now())+';'+name[:-1]+' Exit\n')

In [12]:
Task_list = list(conf.keys())[0]
f = open(fname[:-5]+"_Log.txt", "w")
f.write(str(datetime.now())+';'+Task_list+' Entry\n')
Activities_list = conf[Task_list]['Activities']
for i in Activities_list:
    runTask(Activities_list[i],[Task_list,i])

f.write(str(datetime.now())+';'+Task_list+' Exit')
f.close()

{'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskA_Input', 'ExecutionTime': '1'}}
1
1
{'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskB_Input', 'ExecutionTime': '2'}}
1
2
{'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskC': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskC_Input', 'ExecutionTime': '3'}}, 'TaskD': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskD_Input', 'ExecutionTime': '4'}}}}
{'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskC_Input', 'ExecutionTime': '3'}}
1
3
{'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskD_Input', 'ExecutionTime': '4'}}
1
4
